In [1]:
# import numpy as np
import pandas as pd
import struct

In [2]:
FILENAME = 'txnlog.dat'
USERID = 2456938384156277127

Header (9 total) 4 byte magic string "MPS7" | 1 byte version | 4 byte (uint32)  of records 


record type chart:

0x00: Debit b'x00'

0x01: Credit b'\x01'

0x02: StartAutopay b'\x02' 

0x03 : EndAutopay b'\x03'


record is 13 bytes dollar amount is 8 byte (float64) amount in dollars

| 1 byte record type enum | 4 byte (uint32) Unix timestamp | 8 byte (uint64) user ID |


In [3]:
with open(FILENAME, 'rb') as binary:
  txns = binary.read()

# unpack header
magic_string = struct.unpack('>4s', txns[0:4])
version_no = struct.unpack('>B', txns[4:5])
total_records = struct.unpack('>I', txns[5:9])


# print header for sanity check 
print('Magic String: ', magic_string[0], '\nVersion Number: ', version_no[0], '\nTotal Records: ', total_records[0])


# print first record for sanity check 
first_record = struct.unpack('>cIQd', txns[9:30])
print('first record: ', first_record)


Magic String:  b'MPS7' 
Version Number:  1 
Total Records:  71
first record:  (b'\x00', 1393108945, 4136353673894269217, 604.274335557087)


In [4]:
txn_type = {b'\x00':'Debit', b'\x01':'Credit', b'\x02':'StartAutopay', b'\x03':'EndAutopay'}
header_length = 9
record_length = 13
dollar_length = 8

# Skip header
header_length = 9
# start at first record
start_byte = header_length

In [5]:
df_column_names = ['Transaction_Type', 'Unix_Timestamp', 'User_ID', 'Amount']
all_transactions = pd.DataFrame(columns=df_column_names)

In [6]:
for i in range(total_records[0]):
  stop_byte = start_byte + record_length
  (txns_type, timestamp, user) = struct.unpack('>cIQ', txns[start_byte:stop_byte])
  
  # set start to where last txn stopped
  start_byte = stop_byte 
  transaction_amount = None # records that do not contain an amount

  transaction_type = txn_type[txns_type]


  if transaction_type == 'Debit' or transaction_type == 'Credit':
    stop_byte = start_byte + dollar_length
    transaction_amount = struct.unpack('>d', txns[start_byte:stop_byte])[0]
    start_byte = stop_byte
  
  print(transaction_type, timestamp, user, transaction_amount)
  all_transactions.loc[i] = [transaction_type, timestamp, user, transaction_amount]


Debit 1393108945 4136353673894269217 604.274335557087
Credit 1416458602 1486945396868222946 825.129614841758
Debit 1398140960 1019092597011251431 668.673048640753
StartAutopay 1390539963 3724427934598140041 None
Debit 1416760748 6837272077571506036 397.73876190471094
Credit 1416425125 7979830878773245174 608.1343005023912
StartAutopay 1407728826 169747289235870461 None
Debit 1407057140 6555455521637047278 163.32836050527746
Debit 1416615422 6823688420765684666 384.61670768440615
EndAutopay 1389541294 7178585665953385255 None
Credit 1402457849 3018469034978866138 154.66390447034632
Debit 1405988060 1518491488393966864 882.4780989014131
EndAutopay 1413632673 2477346412368114538 None
Credit 1411685979 3281373847403844559 828.5647201149624
StartAutopay 1416958471 685213522303989579 None
Debit 1405300715 4596876061716608039 654.8593045062225
StartAutopay 1417477182 4011359550169803385 None
Debit 1388954072 6648738534997005833 286.292471682929
EndAutopay 1401299706 7979830878773245174 None
D

In [7]:
all_transactions.head(71)

,Transaction_Type,Unix_Timestamp,User_ID,Amount
0,Debit,1393108945,4136353673894269217,604.274
1,Credit,1416458602,1486945396868222946,825.13
2,Debit,1398140960,1019092597011251431,668.673
3,StartAutopay,1390539963,3724427934598140041,None
4,Debit,1416760748,6837272077571506036,397.739
...,...,...,...,...
66,Credit,1405282673,2456938384156277127,248.585
67,Credit,1412703694,8505906760983331750,955.292
68,Credit,1389850890,2691316960514504584,633.15
69,Credit,1416613150,6710715718024909786,309.839


In [8]:
total_debits = round(all_transactions.loc[all_transactions['Transaction_Type'] == 'Debit', 'Amount'].sum(), 2)
print(total_debits)

18203.7


In [9]:
total_credits = round(all_transactions.loc[all_transactions['Transaction_Type'] == 'Credit', 'Amount'].sum(), 2)
print(total_credits)

9366.02


In [10]:
is_started = all_transactions['Transaction_Type'] == 'StartAutopay'
print(all_transactions[is_started]['Transaction_Type'].count())

10


In [11]:
is_ended = all_transactions['Transaction_Type'] == 'EndAutopay'
print(all_transactions[is_ended]['Transaction_Type'].count())

8


In [12]:
# User ID 2456938384156277127
is_user = all_transactions['User_ID'] == 2456938384156277127

user_transactions = all_transactions[is_user]
user_transactions.head()

user_credits = round(user_transactions.loc[user_transactions['Transaction_Type'] == 'Credit', 'Amount'].sum(), 2)

user_debits = round(user_transactions.loc[user_transactions['Transaction_Type'] == 'Debit', 'Amount'].sum(), 2)

balance_for_user = user_credits - user_debits

print("{:.2f}".format(balance_for_user))


0.00


# Reformat code for readability

In [14]:
def calc_user_balance(txns_to_parse, user_id):
    
    is_user = txns_to_parse['User_ID'] == user_id

    user_transactions = txns_to_parse[is_user]
    user_transactions.head()

    user_credits = round(user_transactions.loc[user_transactions['Transaction_Type'] == 'Credit', 'Amount'].sum(), 2)

    user_debits = round(user_transactions.loc[user_transactions['Transaction_Type'] == 'Debit', 'Amount'].sum(), 2)

    balance_for_user = user_credits - user_debits

    return(balance_for_user)

balance_for_user = calc_user_balance(all_transactions, USERID)
print(f"balance for user 2456938384156277127 = {balance_for_user:.2f}")

balance for user 2456938384156277127 = 0.00


In [15]:
def calc_autopay_ended(txns_to_parse):
    is_ended = txns_to_parse['Transaction_Type'] == 'EndAutopay'
    return(txns_to_parse[is_ended]['Transaction_Type'].count())

is_ended = calc_autopay_ended(all_transactions)
print(f"autopays ended = {is_ended}")

autopays ended = 8


In [18]:
def calc_autopay_started(txns_to_parse):
    is_started = txns_to_parse['Transaction_Type'] == 'StartAutopay'
    return(txns_to_parse[is_started]['Transaction_Type'].count())

is_started = calc_autopay_started(all_transactions)
print(f"autopays started = {is_started}")

autopays started = 10


In [19]:
def calc_total_debits(txns_to_parse):
    total_debits = round(txns_to_parse.loc[txns_to_parse['Transaction_Type'] == 'Debit', 'Amount'].sum(), 2)
    return(total_debits)

total_debits = calc_total_debits(all_transactions)
print(f"total debit amount = {total_debits}")

total debit amount = 18203.7


In [21]:
def calc_total_credits(txns_to_parse):
    total_credits = round(txns_to_parse.loc[txns_to_parse['Transaction_Type'] == 'Credit', 'Amount'].sum(), 2)
    return(total_credits)

total_credits = calc_total_credits(all_transactions)
print(f"total credit amount = {total_credits}")

total credit amount = 9366.02
